# Women Who Code :: Build-Your-Own Dataset

Sometimes data scientists are handed a fully prepared and cleaned dataset, but this is rarely the case. Today's workshop will give you practice in building your own dataset from scratch. We will use public APIs and publically available data files to create a dataset of weather and population data that is ready for downstream uses.

In this workshop, we'll be collecting and organizing information for fictional visitors to our fictional 


In [35]:
from faker import Faker
import pandas as pd
import requests

# Part 1 :: Calling Public APIs

In this first section we will use several publically available APIs to collect information about fictional visitors to our website. The only information we directly collect about visitors is their IP address. Beyond that, we'll have to look to outside sources to pull in information to learn more about our visitors.

## Get your IP address

An IP address is a unique address that identifies a device on the internet or a local network. IP stands for "Internet Protocol," which is the set of rules governing the format of data sent via the internet or local network.

To find out your own IP address, you can make a call to the [ipify](https://www.ipify.org/) API, a simple public IP address API. This API does not require an account or API key.

In [3]:
# api endpoint
url = "https://api.ipify.org"

# request formatted response
params = {
    "format": "json"
}

resp = requests.get(url, params)

Congrats, you just made a call to the first API of this workshop! Let's take a closer look at the response, and see what information we've collected from it.

In [5]:
# http response status codes indicate whether the request has been successfully completed
resp.status_code

200

In [6]:
# can get response in a string format
resp.text

'{"ip":"3.94.219.192"}'

In [7]:
# or more usefully as a json dictionary
resp.json()

{'ip': '3.94.219.192'}

In [8]:
# let's hold on to ip address, and use it in some next steps
ip = resp.json()["ip"]

## Get location for an IP address

IP addresses can be linked to information about the location where you are connected to the internet.

To find geolocation information given an IP address, we can use the [ip-api](https://ip-api.com/) JSON endpoint. The IP address endpoint allowed us to pass the desired response format (JSON) as a query parameter, but this API has a specific JSON endpoint, so we'll specify the data format as part of the URL.

In [9]:
url = f"http://ip-api.com/json/{ip}"

resp = requests.get(url)

In [10]:
# check that the call succeeded
resp.status_code

200

In [11]:
# inspect the returned data
resp.json()

{'status': 'success',
 'country': 'United States',
 'countryCode': 'US',
 'region': 'VA',
 'regionName': 'Virginia',
 'city': 'Ashburn',
 'zip': '20149',
 'lat': 39.0438,
 'lon': -77.4874,
 'timezone': 'America/New_York',
 'isp': 'Amazon Technologies Inc.',
 'org': 'AWS EC2 (us-east-1)',
 'as': 'AS14618 Amazon.com, Inc.',
 'query': '3.94.219.192'}

In [14]:
# pull out the lat/long fields, since we can look up info about this location

lat = resp.json()["lat"]
long = resp.json()["lon"]

In [15]:
def get_location_info(ip: str):
    """
    Given an IP address, return a dictionary of location information.
    """
    url = f"http://ip-api.com/json/{ip}"
    resp = requests.get(url)
    return resp.json()

## Get local weather

Now that we know where a visitor is from, we can collect any information from other sources to understand the user's location. For this workshop, let's say that our website has content related to weather, and so the visitor's current weather is of interest to us.

Given the visitor's latitude and longitude, we can use the [Open Meteo](https://open-meteo.com/en) API to get information about the location's current weather. Like the other APIs we've used in this workshop, Open Meteo is public and does not require an API key.



In [31]:
url = "https://api.open-meteo.com/v1/forecast"

params = {
    "latitude": lat,
    "longitude": long,
    "current_weather": True,
    "format": "json"
}

resp = requests.get(url, params)

In [32]:
resp.status_code

200

In [33]:
resp.json()

{'generationtime_ms': 0.11897087097167969,
 'longitude': -77.5,
 'elevation': 98.3125,
 'current_weather': {'temperature': 22.6,
  'time': '2022-03-18T19:00',
  'winddirection': 183,
  'windspeed': 14.9,
  'weathercode': 3},
 'latitude': 39,
 'utc_offset_seconds': 0}

In [34]:
def get_weather_info(lat: float, long: float):
    """
    Given a latitude and longitude, return the current weather.
    """
    url = "https://api.open-meteo.com/v1/forecast"

    params = {
        "latitude": lat,
        "longitude": long,
        "current_weather": True,
        "format": "json"
    }

    resp = requests.get(url, params)
    return resp.json()


## Generate fake IP addresses

Since we are working with fake data, we'll have to create some fake IP addresses for our website visitors. To 

In [36]:
# example of how Faker works
faker = Faker()
faker.ipv4() 


'102.157.103.246'

In [38]:
# make a list of all our IP addresses
fake_ips = [faker.ipv4() for _ in range(100)]
fake_ips[0:5]

['82.143.16.5',
 '96.5.183.7',
 '59.248.109.20',
 '79.122.136.94',
 '81.238.19.228']

In [44]:
def get_geo_weather_data(ip: str):
    
    # get location info for the ip address
    location_info = get_location_info(ip)
    
    # get the current weather at the lat/long
    weather_info = get_weather_info(location_info["lat"], location_info["lon"])
    
    # stack the dictionaries
    # only works in python 3.9 and above
    all_data = {"ip": ip} | location_info | weather_info
    
    # alternative for lower python versions using unpacking
    # all_data = {**ip_info, **location_info, **weather_info}
    
    return all_data

In [47]:
get_geo_weather_data(fake_ips[0])

{'ip': '82.143.16.5',
 'status': 'success',
 'country': 'Italy',
 'countryCode': 'IT',
 'region': '25',
 'regionName': 'Lombardy',
 'city': 'Milan',
 'zip': '20121',
 'lat': 45.4642,
 'lon': 9.18998,
 'timezone': 'Europe/Rome',
 'isp': 'ip telecom srl',
 'org': 'TPN',
 'as': 'AS29449 ip telecom srl',
 'query': '82.143.16.5',
 'latitude': 45.46,
 'longitude': 9.18,
 'elevation': 132.625,
 'current_weather': {'time': '2022-03-18T20:00',
  'weathercode': 3,
  'winddirection': 77,
  'windspeed': 10.8,
  'temperature': 10.6},
 'utc_offset_seconds': 0,
 'generationtime_ms': 0.2110004425048828}

# Generate Dataset

In [30]:
weather_dicts = [get_fake_geo_weather_data() for i in range(100)]

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
df_weather = pd.DataFrame(weather_dicts)

In [ ]:
df_weather.head()

In [ ]:
df_weather["country"].unique()

# Join with Migration Data

XLSX file available from the UN

https://population.un.org/wpp/Download/Standard/Migration/

In [ ]:
df_migration = pd.read_excel("WPP2019_MIGR_F01_NET_MIGRATION_RATE.xlsx", skiprows=range(16))

In [ ]:
df_migration.head()

In [ ]:
df_migration.tail()

In [ ]:
# limit to only country-level records
df_migration_countries = df_migration.loc[df_migration["Type"]=="Country/Area"]

df_migration_countries.head()

In [ ]:
df_migration_subset = df_migration_countries[["Region, subregion, country or area *", "2015-2020"]]

In [ ]:
df_migration_subset.head()

In [ ]:
df_migration_subset.rename({"Region, subregion, country or area *": "country", "2015-2020": "migration_rate"}, axis=1, inplace=True)

In [ ]:
df_migration_subset.head()

In [ ]:
df_weather_migration = df_weather.merge(df_migration_subset, how="left", left_on="country", right_on="country")

In [ ]:
df_weather_migration.head()

In [ ]:
# some countries didn't match because names are different

df_weather_migration.loc[df_weather_migration["migration_rate"].isna(), "country"].unique()

In [ ]:
# a few countries did match up

df_weather_migration.loc[df_weather_migration["migration_rate"].notna(), "country"].unique()

In [ ]:
df_migration_subset.loc[df_migration_subset["country"].str.contains("United States")]

In [ ]:
df_migration_subset.loc[df_migration_subset["country"].str.contains("Syria")]

In [ ]:
df_migration_subset.loc[df_migration_subset["country"].str.contains("Korea")]

In [ ]:
df_migration_subset.loc[df_migration_subset["country"].str.contains("Russia")]

In [ ]:
df_migration_subset.loc[df_migration_subset["country"].str.contains("Venezuela")]

In [ ]:
df_migration_subset.loc[df_migration_subset["country"].str.contains("Viet")]

In [ ]:
df_migration_subset.loc[df_migration_subset["country"].str.contains("Taiwan")]

In [ ]:
# to get join to work, let's rename country in the migration dataset

df_migration_subset["country"].replace(to_replace={
    "United States of America": "United States", 
    "Syrian Arab Republic": "Syria", 
    "Russian Federation": "Russia", 
    "Republic of Korea": "South Korea",
    "Venezuela (Bolivarian Republic of)": "Venezuela",
    "Viet Nam": "Vietnam",
    "China, Taiwan Province of China": "Taiwan",
    
}, inplace=True)

In [ ]:
# try the join again
df_weather_migration = df_weather.merge(df_migration_subset, how="left", left_on="country", right_on="country")

In [ ]:
df_weather_migration.head()

In [ ]:
# now all of the records should have a match

df_weather_migration.loc[df_weather_migration["migration_rate"].isna(), "country"].unique()